# Imports - TO DO add helpers, streamline process

In [2]:
import pandas as pd
import os
import re
import numpy as np
import ast


# Classify File Instances

In [ ]:
file_path = "../outputs/persistent_ce/persistent_attack_ce.csv"
dir_path = "../outputs/persistent_ce"

def classify_simulations(files):
    ce_reg = r'.*\d_cex.*'
    nc_reg = r'.*no_cex.*'
    att_reg = r".*attacker_crash.*"

    reg = [ce_reg,nc_reg,att_reg]
    classified_cases = [[],[],[]]
    for simulation in files:
        for i in range(len(reg)):
            found = re.match(reg[i],simulation)
            if found:
                classified_cases[i].append(simulation)
                break
    return classified_cases

def read_folder(folder,sampler_file, attack_type, velocity_file):    
    files = os.listdir(folder)
    final = classify_simulations(files)

    sum = 0
    for i in final:
        sum += len(i)
        print(len(i))

    #sanity check
    print(sum == len(files)-2) #account for the velocities and sampler files 

    simulation_params_df = pd.read_csv(sampler_file)

    if attack_type == "int":
        return(preprocess_intermittent(final,simulation_params_df,velocity_file))
    if attack_type == "per":
        return(preprocess_persistent(final,simulation_params_df,velocity_file))
    else:
        print("Error classifying attack type")


def preprocess_intermittent(classified_files, sampler_data,velocity_file):
    intermittent_cols = ["point.params.random_thr_br1","point.params.random_thr_br2","point.params.inter_vehicle_distance", "point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4","point.params.random_thr_br1","rho"]
    sampler_data =sampler_data[intermittent_cols]
    output = get_attack_spacing_intermittent(sampler_data)
    output = velocity_at_attack_intermittent(sampler_data,velocity_file)

    return output


def preprocess_persistent(classified_files,sampler_data):
    pass


# Selected params based on Attack type

Persistent_attack uses frequency to define how often the attack is applied. What are some interesting feature we can extract from this? 
 Total attacks?
 Time between attacks?
 Velocity at attack time would still be interesting just a bit more complex to extract.


Likely need to rerun data as there was a useless variable EGO_BRAKING_THESHOLD being included in the search space.. I am not sure how much of an effect this would have had but can always re run and see what happens

In [10]:
file_path = "../outputs/persistent_ce/persistent_attack_ce.csv"
dir_path = "../outputs/persistent_ce"

simulation_params_df = pd.read_csv("../outputs/persistent_ce/persistent_attack_ce.csv")

In [11]:
persistent_cols = [ "point.params.amplitude_acc", "point.params.attack_time", "point.params.frequency", "point.params.inter_vehicle_distance", "rho"]
intemittent_cols = ["point.params.random_thr_br1","point.params.random_thr_br2","point.params.inter_vehicle_distance", "point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4","point.params.random_thr_br1","rho"]

In [12]:
evaluation_simulation_params_df = simulation_params_df[persistent_cols]
print(len(evaluation_simulation_params_df))

500


In [14]:
def get_attack_spacing_intermittent(intermittent_df): # Calculate distance between each attack and add to Dataframe
    attack_spacing = []

    for simulation_instance in range(len(intermittent_df)):
        diff_1 = intermittent_df["point.params.random_t2"][simulation_instance] - intermittent_df["point.params.random_t1"][simulation_instance] # t2 > t1
        diff_2 = intermittent_df["point.params.random_t3"][simulation_instance] - intermittent_df["point.params.random_t2"][simulation_instance]
        diff_3 = intermittent_df["point.params.random_t4"][simulation_instance] - intermittent_df["point.params.random_t3"][simulation_instance]

        sim_gaps = np.array([diff_1,diff_2,diff_3])
        attack_spacing.append(sim_gaps)

    attack_spacing = np.array(attack_spacing)
    col_names = ["t1_t2", "t2_t3", "t3_t4"]
    time_df = pd.DataFrame(attack_spacing, columns=col_names)

    return(pd.concat([intermittent_df, time_df],axis=1))



def velocity_at_attack_intermittent(velocities_df, intermittent_df): # Collect the attackers speed at each attack and append to Dataframe
    simulation_velocities_arr = []
    velocities_df = velocities_df.reset_index(drop=True)
    
    for j in range(0,len(velocities_df)):
        sim_velocities = []
        for i in velocities_df.columns:
            vals = ast.literal_eval(velocities_df[i][j])
            velocities = [i[1] for i in vals]
            sim_velocities.append(velocities)
        
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance

    simulation_velocities_arr = np.array(simulation_velocities_arr)

    
    col_names = ["t1_velocity", "t2_velocity", "t3_velocity", "t4_velocity"]
    velocity_df = pd.DataFrame(simulation_velocities_arr, columns=col_names)

    return(pd.concat([intermittent_df, velocity_df],axis=1))



In [50]:
evaluation_simulation_params_df = get_attack_time_diff(evaluation_simulation_params_df)

In [ ]:
evaluation_simulation_params_df

# Isolate different crash instances

In [8]:
evaluation_simulation_params_ce_df = evaluation_simulation_params_df[:len(final[0])]
non_ce_idx = len(final[1]) + len(final[2])
evaluation_simulation_params_nce_df = evaluation_simulation_params_df[:non_ce_idx]

NameError: name 'evaluation_simulation_params_df' is not defined

In [162]:
print(len(evaluation_simulation_params_df) == len(evaluation_simulation_params_ce_df)+ len(evaluation_simulation_params_nce_df))

True


# Comparing Attributes Across sets
 Need to crate function for sepearting no crash, and attacker crash instances for analysis

In [9]:
evaluation_simulation_params_ce_df.describe()


,point.params.EGO_BRAKING_THRESHOLD,point.params.inter_vehicle_distance,point.params.random_t1,point.params.random_t2,point.params.random_t3,point.params.random_t4,point.params.random_thr_br1,rho
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,8.713029,9.746584,14.209324,27.643985,35.297768,46.427851,0.161618,-0.228502
std,2.963695,0.972200,2.646077,2.179032,2.071668,3.285723,0.201605,0.034751
min,5.106673,7.522011,10.461696,23.207072,31.539357,41.114363,0.000577,-0.252136
25%,6.104825,9.013499,11.627715,25.642673,34.005609,43.669777,0.041240,-0.245407
50%,8.886217,9.777100,14.556311,28.285041,35.536989,48.285184,0.100297,-0.240379
75%,10.510309,10.530265,16.743642,29.448682,36.425461,49.177634,0.189451,-0.235022
max,14.860658,11.522309,17.352173,29.925453,39.205542,49.931584,0.751167,-0.119553


In [10]:
evaluation_simulation_params_nce_df.describe()

,point.params.EGO_BRAKING_THRESHOLD,point.params.inter_vehicle_distance,point.params.random_t1,point.params.random_t2,point.params.random_t3,point.params.random_t4,point.params.random_thr_br1,rho
count,978.000000,978.000000,978.000000,978.000000,978.000000,978.000000,978.000000,978.000000
mean,7.383048,9.761085,13.215801,26.570605,36.873998,48.746901,0.124762,0.766189
std,2.205238,2.825905,1.566623,1.631296,1.356026,1.482323,0.135520,2.576300
min,5.003328,7.003368,10.018098,21.129501,31.044543,41.138741,0.000033,-0.281975
25%,5.696318,8.166914,12.349983,25.340170,36.632223,48.563829,0.047859,-0.138528
50%,6.455193,9.315110,13.043298,26.586797,37.160300,49.041234,0.100098,0.239134
75%,9.506111,10.611565,13.777675,27.738099,37.712347,49.529489,0.159088,0.248223
max,14.984047,29.965331,19.756060,29.999612,39.964084,49.996488,0.990331,22.763672


In [11]:
for i in intemittent_cols:
    stats= []
    stats.append(evaluation_simulation_params_ce_df.describe())
    print(evaluation_simulation_params_ce_df.describe()[i], evaluation_simulation_params_nce_df.describe()[i])

count    22.000000
mean      8.713029
std       2.963695
min       5.106673
25%       6.104825
50%       8.886217
75%      10.510309
max      14.860658
Name: point.params.EGO_BRAKING_THRESHOLD, dtype: float64 count    978.000000
mean       7.383048
std        2.205238
min        5.003328
25%        5.696318
50%        6.455193
75%        9.506111
max       14.984047
Name: point.params.EGO_BRAKING_THRESHOLD, dtype: float64
count    22.000000
mean      9.746584
std       0.972200
min       7.522011
25%       9.013499
50%       9.777100
75%      10.530265
max      11.522309
Name: point.params.inter_vehicle_distance, dtype: float64 count    978.000000
mean       9.761085
std        2.825905
min        7.003368
25%        8.166914
50%        9.315110
75%       10.611565
max       29.965331
Name: point.params.inter_vehicle_distance, dtype: float64
count    22.000000
mean     14.209324
std       2.646077
min      10.461696
25%      11.627715
50%      14.556311
75%      16.743642
max      17.3

In [12]:
ce_ranges = {}
nce_ranges = {}
for i in intemittent_cols:
    ce_ranges[i] = np.max(evaluation_simulation_params_ce_df[i]) - np.min(evaluation_simulation_params_ce_df[i])
    nce_ranges[i] = np.max(evaluation_simulation_params_nce_df[i]) - np.min(evaluation_simulation_params_nce_df[i])
    print(np.max(evaluation_simulation_params_ce_df[i]), np.min(evaluation_simulation_params_ce_df[i]))

14.860658480180678 5.1066730583735165
11.522308561642983 7.522011090729293
17.352172991394518 10.461696033802358
29.92545343586779 23.20707233727357
39.205541923029074 31.539356729392782
49.93158373976293 41.11436279334789
0.7511673005702715 0.0005768858600884
-0.1195526125093415 -0.2521362119258663


In [13]:
for i in intemittent_cols:
    print(f"Counter-example range for category {i}: {ce_ranges[i]}, non-counter example range: {nce_ranges[i]}")
    print(f"Category diff is: {np.abs(ce_ranges[i] - nce_ranges[i])}")

Counter-example range for category point.params.EGO_BRAKING_THRESHOLD: 9.753985421807162, non-counter example range: 9.980718524651893
Category diff is: 0.22673310284473125
Counter-example range for category point.params.inter_vehicle_distance: 4.00029747091369, non-counter example range: 22.961962636790023
Category diff is: 18.961665165876333
Counter-example range for category point.params.random_t1: 6.89047695759216, non-counter example range: 9.737961754091387
Category diff is: 2.847484796499227
Counter-example range for category point.params.random_t2: 6.718381098594222, non-counter example range: 8.87011135496638
Category diff is: 2.151730256372158
Counter-example range for category point.params.random_t3: 7.6661851936362915, non-counter example range: 8.91954048236823
Category diff is: 1.2533552887319388
Counter-example range for category point.params.random_t4: 8.817220946415041, non-counter example range: 8.857747223274195
Category diff is: 0.04052627685915411
Counter-example r

In [14]:
evaluation_simulation_params_nce_df

,point.params.EGO_BRAKING_THRESHOLD,point.params.inter_vehicle_distance,point.params.random_t1,point.params.random_t2,point.params.random_t3,point.params.random_t4,point.params.random_thr_br1,rho
22,10.711557,9.314857,13.954744,29.301319,33.577055,49.862408,0.225958,-0.257583
23,10.794390,7.172056,13.365356,29.886375,37.881430,49.540354,0.330457,-0.236824
24,5.758931,9.188924,13.571831,26.508706,35.479573,49.969510,0.133132,-0.243393
25,10.064621,10.180070,13.236471,28.950187,34.716427,48.377991,0.028232,-0.239952
26,9.687717,11.453984,14.373270,29.840049,36.202533,48.436736,0.083296,-0.252987
...,...,...,...,...,...,...,...,...
995,5.440200,8.214613,13.400264,27.044562,36.673028,48.333943,0.159250,0.240692
996,6.783876,10.321772,13.602919,25.035343,37.420631,48.718263,0.127937,0.460510
997,6.797245,7.841638,13.180970,24.834107,38.116130,49.061100,0.089367,0.240674
998,6.747919,8.241540,12.497683,24.772646,36.800799,49.838133,0.127814,0.237396


In [15]:
evaluation_simulation_params_nce_df.reset_index(drop=True, inplace=True)


 It seems that t1 attack, initial vehicle distance, and ego braking threshold are most significant in forcing a crash. These categories have the most range in between the two sets. (Counter-examples and non
)

In [16]:
examples = []

for i in range(0,len(evaluation_simulation_params_nce_df)):
    if np.abs(np.mean(evaluation_simulation_params_ce_df["point.params.inter_vehicle_distance"]) -
               evaluation_simulation_params_nce_df["point.params.inter_vehicle_distance"][i]) < .5:
      # print(evaluation_simulation_params_nce_df["point.params.inter_vehicle_distance"][i] - np.mean(evaluation_simulation_params_ce_df["point.params.inter_vehicle_distance"]))
        examples.append(evaluation_simulation_params_nce_df.iloc[i])

In [64]:
print(len(examples))

193


In [18]:
examples = pd.DataFrame(examples)

In [19]:
print(len(examples))

193


In [20]:
examples.describe()

,point.params.EGO_BRAKING_THRESHOLD,point.params.inter_vehicle_distance,point.params.random_t1,point.params.random_t2,point.params.random_t3,point.params.random_t4,point.params.random_thr_br1,rho
count,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000
mean,6.952226,9.744507,13.007916,26.327024,37.090970,48.978556,0.100100,-0.029217
std,1.861507,0.300492,1.243267,1.403338,0.978324,1.026812,0.089156,0.537032
min,5.014987,9.250714,10.042208,21.563227,31.199958,41.397373,0.000050,-0.281975
25%,5.623539,9.482965,12.348164,25.182132,36.730940,48.643257,0.044768,-0.240269
50%,6.285440,9.756275,12.964844,26.348285,37.195090,49.115231,0.084553,-0.220383
75%,6.960798,10.001100,13.607577,27.367353,37.689864,49.544296,0.138487,0.055326
max,12.950636,10.224737,19.096618,29.519233,39.115091,49.996488,0.860559,4.726166


In [21]:
evaluation_simulation_params_ce_df.describe()

,point.params.EGO_BRAKING_THRESHOLD,point.params.inter_vehicle_distance,point.params.random_t1,point.params.random_t2,point.params.random_t3,point.params.random_t4,point.params.random_thr_br1,rho
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,8.713029,9.746584,14.209324,27.643985,35.297768,46.427851,0.161618,-0.228502
std,2.963695,0.972200,2.646077,2.179032,2.071668,3.285723,0.201605,0.034751
min,5.106673,7.522011,10.461696,23.207072,31.539357,41.114363,0.000577,-0.252136
25%,6.104825,9.013499,11.627715,25.642673,34.005609,43.669777,0.041240,-0.245407
50%,8.886217,9.777100,14.556311,28.285041,35.536989,48.285184,0.100297,-0.240379
75%,10.510309,10.530265,16.743642,29.448682,36.425461,49.177634,0.189451,-0.235022
max,14.860658,11.522309,17.352173,29.925453,39.205542,49.931584,0.751167,-0.119553


# Collecting Extra Scenic data to construct new features for analysis

In [ ]:
def velocity_at_attack_intermittent(velocities_df, intermittent_df):
    simulation_velocities_arr = []
    velocities_df = velocities_df.reset_index(drop=True)
    
    for j in range(0,len(velocities_df)):
        sim_velocities = []
        for i in velocities_df.columns:
            vals = ast.literal_eval(velocities_df[i][j])
            velocities = [i[1] for i in vals]
            sim_velocities.append(velocities)
        
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance

    simulation_velocities_arr = np.array(simulation_velocities_arr)

    
    col_names = ["t1_velocity", "t2_velocity", "t3_velocity", "t4_velocity"]
    velocity_df = pd.DataFrame(simulation_velocities_arr, columns=col_names)

    return(pd.concat([intermittent_df, velocity_df],axis=1))

    

In [100]:
simulation_velocities_df = pd.read_csv("intermittent_attack.scenic1000_velocities.csv")
simulation_velocities_df = simulation_velocities_df.reset_index(drop=True)

In [ ]:
simulation_velocities_df

simulation_velocities_arr = []


for j in range(0,len(simulation_velocities_df)):
    sim_velocities = []
    for i in simulation_velocities_df.columns:
        vals = ast.literal_eval(simulation_velocities_df[i][j])
        velocities = [i[1] for i in vals]
        sim_velocities.append(velocities)
    
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance



# Gathering velocity at the time of attack

In [ ]:
attack_times = evaluation_simulation_params_df[["point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4"]]

Velocity is measured at each timestep with is 0.1 or one tenth of a second. We need to account for this when we consider the velocity before the attacker slams on the breaks. 

In [ ]:
velocity_at_attack = []
for i in range(len(attack_times)):
    sim_attack = []
    for j in attack_times:
        idx = np.floor(attack_times[j][i]) * 10
        if idx > len(simulation_velocities_arr[i][0]): #check that the sim didn't end before the attack was instanciated
            sim_attack.append(0)
        else:
            sim_attack.append(simulation_velocities_arr[i][0][int(idx)]) 
    velocity_at_attack.append(np.array(sim_attack))


In [ ]:
len(velocity_at_attack)
for i in range(5):
    print(velocity_at_attack[i])

# More Stuff
